In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Embedding, LSTM, Dense, Dropout, Input
from keras.preprocessing import sequence
from keras.preprocessing.text import one_hot, text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.datasets import imdb
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from sklearn.metrics import confusion_matrix

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size ch

In [2]:
data1 = pd.read_csv('dev.txt', index_col=0, delimiter='\t')

In [3]:
data2 = pd.read_csv('train.txt', index_col=0, delimiter='\t')

In [4]:
data = pd.concat([data1, data2])

In [5]:
# data3 = pd.read_csv('test.txt', index_col=0, delimiter='\t')

In [6]:
data.head()

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
ID,,,,,,,,,,,,
2018-En-00866,"@RanaAyyub @rajnathsingh Oh, hidden revenge an...",1,0,1,0,0,0,0,0,0,0,0
2018-En-02590,I'm doing all this to make sure you smiling do...,0,0,0,0,1,1,1,0,0,0,0
2018-En-03361,if not then #teamchristine bc all tana has don...,1,0,1,0,0,0,0,0,0,0,0
2018-En-03230,It is a #great start for #beginners to jump in...,0,0,0,0,1,0,1,0,0,0,0
2018-En-01143,My best friends driving for the first time wit...,0,0,0,1,0,0,0,0,0,0,0


In [7]:
data.iloc[:,0] = data.iloc[:,0].apply(lambda x: re.sub('@\w+', ' ', x))

In [8]:
data.iloc[:,0] = data.iloc[:,0].apply(lambda x: re.sub('[^A-Za-z1-9!? ]', ' ', x))

In [9]:
data.iloc[:,0] = data.iloc[:,0].apply(lambda x: x.lower())

In [10]:
#nltk.download('punkt')

In [11]:
stop_words = set(stopwords.words('english'))

In [12]:
data.iloc[:,0] = data.iloc[:,0].apply(lambda x: word_tokenize(x))

In [13]:
data.iloc[:,0] = data.iloc[:,0].apply(lambda x: [w for w in x if not w in stop_words])

In [14]:
data.iloc[:,0] = data.iloc[:,0].apply(lambda x: ' '.join(x))

In [53]:
def preprocess(tweet):
    tweet  = re.sub('@\w+', ' ', tweet)
    tweet  = re.sub('[^A-Za-z1-9!? ]', ' ', tweet)
    tweet = tweet.lower()
    stop_words = set(stopwords.words('english'))
    tweet = word_tokenize(tweet)
    tweet  = [w for w in tweet if not w in stop_words]
    tweet  = ' '.join(tweet)
    tweet  = one_hot(tweet, 3000, lower=False)
    processed = pad_sequences([tweet], 35, padding='post', truncating='post')
    return processed

In [15]:
data.head()

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
ID,,,,,,,,,,,,
2018-En-00866,oh hidden revenge anger rememberthe time rebutted,1,0,1,0,0,0,0,0,0,0,0
2018-En-02590,make sure smiling bro,0,0,0,0,1,1,1,0,0,0,0
2018-En-03361,teamchristine bc tana done provoke tweeting sh...,1,0,1,0,0,0,0,0,0,0,0
2018-En-03230,great start beginners jump auto trading profit...,0,0,0,0,1,0,1,0,0,0,0
2018-En-01143,best friends driving first time car terrifying,0,0,0,1,0,0,0,0,0,0,0


In [16]:
sentences = []
for tweet in data.iloc[:,0]:
    sentences.append((one_hot(tweet, 3000, lower=False)))

In [17]:
sentences

[[793, 2425, 616, 1259, 122, 2686, 593],
 [1794, 2113, 1772, 2618],
 [1139, 247, 663, 886, 2773, 225, 539, 314, 1762, 981, 998, 2984, 115],
 [634,
  752,
  1566,
  731,
  2835,
  951,
  748,
  684,
  936,
  1817,
  2393,
  1414,
  143,
  700,
  820,
  2172,
  1414],
 [2410, 1587, 1276, 2464, 2686, 949, 1549],
 [2134,
  80,
  2928,
  1817,
  2581,
  447,
  2296,
  1978,
  2739,
  719,
  2495,
  1249,
  2189,
  719,
  1773,
  447,
  2784],
 [2237, 2436, 2938, 2464, 2967, 469, 1171, 1920, 1801, 2857, 550, 206],
 [2898, 1641, 2113, 919, 1144, 1091, 2118, 1108, 436, 787, 1727],
 [2113, 2212, 2686, 861, 2152, 1144, 2992, 2532, 17, 12],
 [2503, 2234],
 [1239, 164, 1646, 2038, 976, 158, 11],
 [996, 1593, 615, 1518, 2119, 825, 2784, 787, 825, 171],
 [892,
  978,
  2360,
  2763,
  2018,
  1396,
  816,
  1682,
  2285,
  13,
  1217,
  83,
  2264,
  419,
  158],
 [83,
  2148,
  1479,
  771,
  572,
  1244,
  192,
  747,
  788,
  1349,
  466,
  2451,
  1518,
  752,
  875,
  1118],
 [565,
  2664,
  11

In [18]:
data.head()

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
ID,,,,,,,,,,,,
2018-En-00866,oh hidden revenge anger rememberthe time rebutted,1,0,1,0,0,0,0,0,0,0,0
2018-En-02590,make sure smiling bro,0,0,0,0,1,1,1,0,0,0,0
2018-En-03361,teamchristine bc tana done provoke tweeting sh...,1,0,1,0,0,0,0,0,0,0,0
2018-En-03230,great start beginners jump auto trading profit...,0,0,0,0,1,0,1,0,0,0,0
2018-En-01143,best friends driving first time car terrifying,0,0,0,1,0,0,0,0,0,0,0


In [19]:
X = sentences
Y = data.iloc[:,1:]

In [20]:
Y.head()

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
ID,,,,,,,,,,,
2018-En-00866,1,0,1,0,0,0,0,0,0,0,0
2018-En-02590,0,0,0,0,1,1,1,0,0,0,0
2018-En-03361,1,0,1,0,0,0,0,0,0,0,0
2018-En-03230,0,0,0,0,1,0,1,0,0,0,0
2018-En-01143,0,0,0,1,0,0,0,0,0,0,0


In [21]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [22]:
anger_train = y_train['anger']
anticipation_train = y_train['anticipation']
disgust_train = y_train['disgust']
fear_train = y_train['fear']
joy_train = y_train['joy']
love_train = y_train['love']
optimism_train = y_train['optimism']
pessimism_train = y_train['pessimism']
sadness_train = y_train['sadness']
surprise_train = y_train['surprise']
trust_train = y_train['trust']

In [23]:
X = pad_sequences(x_train,35, padding='post', truncating='post')

In [24]:
X[0].shape

(35,)

In [26]:
inputs = Input(shape=(35,))
x = (Embedding(3000, 70))(inputs)
x = LSTM(64, return_sequences=True)(x)
x = Dropout(0.4)(x)
x = LSTM(256, return_sequences=False)(x)
x = Dropout(0.3)(x)
x = Dense(60, activation='relu')(x)
x = Dropout(0.2)(x)
output1  = Dense(1, activation='sigmoid')(x)
output2 = Dense(1, activation='sigmoid')(x)
output3 = Dense(1, activation='sigmoid')(x)
output4  = Dense(1, activation='sigmoid')(x)
output5 = Dense(1, activation='sigmoid')(x)
output6 = Dense(1, activation='sigmoid')(x)
output7 = Dense(1, activation='sigmoid')(x)
output8 = Dense(1, activation='sigmoid')(x)
output9  = Dense(1, activation='sigmoid')(x)
output10 = Dense(1, activation='sigmoid')(x)
output11 = Dense(1, activation='sigmoid')(x)

model = Model(input=inputs, output = [output1,output2, output3, output4, output5, output6, output7, output8, output9, output10, output11])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 35)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 35, 70)       210000      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 35, 64)       34560       embedding_2[0][0]                
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 35, 64)       0           lstm_2[0][0]                     
__________________________________________________________________________________________________
lstm_3 (LS

In [27]:
model.fit(X, [anger_train, anticipation_train, disgust_train, fear_train, joy_train, love_train, optimism_train, pessimism_train, sadness_train, surprise_train, trust_train], epochs=15)

Epoch 1/15
6179/6179 [==============================] - 24s 4ms/step - loss: 5.4672 - dense_2_loss: 0.6832 - dense_3_loss: 0.4398 - dense_4_loss: 0.6868 - dense_5_loss: 0.5011 - dense_6_loss: 0.6801 - dense_7_loss: 0.3635 - dense_8_loss: 0.6297 - dense_9_loss: 0.3858 - dense_10_loss: 0.6251 - dense_11_loss: 0.2297 - dense_12_loss: 0.2425 - dense_2_acc: 0.6020 - dense_3_acc: 0.8529 - dense_4_acc: 0.5889 - dense_5_acc: 0.8174 - dense_6_acc: 0.5936 - dense_7_acc: 0.8948 - dense_8_acc: 0.6956 - dense_9_acc: 0.8775 - dense_10_acc: 0.6846 - dense_11_acc: 0.9472 - dense_12_acc: 0.9469
Epoch 2/15
6179/6179 [==============================] - 20s 3ms/step - loss: 5.2803 - dense_2_loss: 0.6702 - dense_3_loss: 0.4224 - dense_4_loss: 0.6746 - dense_5_loss: 0.4777 - dense_6_loss: 0.6695 - dense_7_loss: 0.3452 - dense_8_loss: 0.6139 - dense_9_loss: 0.3688 - dense_10_loss: 0.6183 - dense_11_loss: 0.2055 - dense_12_loss: 0.2142 - dense_2_acc: 0.6153 - dense_3_acc: 0.8552 - dense_4_acc: 0.6030 - dense_5

Epoch 15/15
6179/6179 [==============================] - 20s 3ms/step - loss: 3.3411 - dense_2_loss: 0.2682 - dense_3_loss: 0.3703 - dense_4_loss: 0.3352 - dense_5_loss: 0.3678 - dense_6_loss: 0.2902 - dense_7_loss: 0.1842 - dense_8_loss: 0.3892 - dense_9_loss: 0.2799 - dense_10_loss: 0.4867 - dense_11_loss: 0.1968 - dense_12_loss: 0.1726 - dense_2_acc: 0.9061 - dense_3_acc: 0.8553 - dense_4_acc: 0.8726 - dense_5_acc: 0.8328 - dense_6_acc: 0.8848 - dense_7_acc: 0.9304 - dense_8_acc: 0.8236 - dense_9_acc: 0.8809 - dense_10_acc: 0.7559 - dense_11_acc: 0.9502 - dense_12_acc: 0.9476A: 15s - loss: 3.2828 - dense_2_loss: 0.2674 - dense_3_loss: 0.3504 - dense_4_loss: 0.3469 - dense_5_loss: 0.3651 - dense_6_loss: 0.2827 - dense_7_loss: 0.1715 - dense_8_loss: 0.3994 - dense_9_loss: 0.2712 - dense_10_loss: 0.4634 - dense_11_loss: 0.1834 - dense_12_loss: 0.1814 - dense


In [30]:
x_test_pad = pad_sequences(x_test, 35, padding='post', truncating='post')

In [41]:
(x_test_pad[0].reshape(1,-1)).shape

(1, 35)

In [44]:
X[0]

array([2008, 1065, 2978,  692, 2342,  452, 1184, 2711, 1048,  508,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0], dtype=int32)

In [42]:
pred = model.predict(x_test_pad[0].reshape(1,-1))

In [54]:
test_tweet  = "This is super bullshit and I hate it, and I am afraid, fuck"

In [55]:
tweet  = preprocess(test_tweet)

In [56]:
tweet

array([[1261, 2442, 2119,  294, 2661,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0]], dtype=int32)

In [57]:
model.predict(tweet)

[array([[0.92934805]], dtype=float32),
 array([[0.04686974]], dtype=float32),
 array([[0.8672546]], dtype=float32),
 array([[0.1465483]], dtype=float32),
 array([[0.05579642]], dtype=float32),
 array([[0.0025526]], dtype=float32),
 array([[0.03108483]], dtype=float32),
 array([[0.1175594]], dtype=float32),
 array([[0.41740823]], dtype=float32),
 array([[0.03453017]], dtype=float32),
 array([[0.00262298]], dtype=float32)]

In [ ]:
cm = confusion_matrix(y_test, pred)

In [ ]:
cm

In [29]:
model.save('LSTMmultiv1.h5')

In [ ]:
np.savetxt("x_test.csv", x_test_pad, delimiter=",")
np.savetxt("x_train.csv", X, delimiter=",")
np.savetxt("y_train.csv", y_train, delimiter=",")
np.savetxt("y_test.csv", y_test, delimiter=",")